# Cálculo de Tasa de Defunción seleccionada por zona metropolitana
## **Instrucciones:**
Cambia el directorio del conjunto de datos y el directorio de trabajo en el primer bloque. Para conocer las opciones de defunciones en lista mexicana debes de revisar elconjunto de defunciones registradas - catalogos - lista_mexicana.csv

## **Links de descarga:**
**Estadísticas de Defunciones Registradas:** https://www.inegi.org.mx/programas/edr/#datos_abiertos
**Crecimiento Demográfico:** https://www.gob.mx/cms/uploads/attachment/file/918028/BD_municipales_portada_regiones_FINAL.pdf
**Características de Metrópolis:** https://datos.gob.mx/dataset/metropolis_mexico_2020/resource/622497d1-7a92-43e1-8279-e8c286889509

In [7]:
# ================================================================
# 0) Setup: rutas, parámetros, lecturas (solo aquí)
# ================================================================

import pandas as pd
from pathlib import Path

# -------------------------
# Base repo (según tu estructura)
# -------------------------
DIR_REPO = Path(r"D:\Contenido\2025_12\Suicidios_Navidad\Repositorio")
DIR_DATA = DIR_REPO / "Data"
DIR_OUTPUT = DIR_REPO / "Output"

# -------------------------
# EDR
# -------------------------
Dir_Defunciones = Path(r"D:\Data Qgis\México\Defuncciones\conjunto_de_datos_edr2024_csv (1)\conjunto_de_datos")
ARCHIVO_CONJ_DEF = "conjunto_de_datos_defunciones_registradas24_csv.csv"

# -------------------------
# Metro
# -------------------------
Dir_Metropolis = DIR_DATA / "Metro"
ARCHIVO_Metropoli_Bridge = "Bridge_ZonasMetro.csv"
ARCHIVO_Carateristicas_Metro = "metropolis_caracteristicas.csv"
ARCHIVO_Proyecciones_Demograficas = "Indicadores_Dem_Mun.csv"

# -------------------------
# Catálogos
# -------------------------
DIR_CATALOGOS = DIR_DATA / "EDR_Catalogos"
CAT_PAISES = "paises.csv"
CAT_OCUPACION = "ocupacion.csv"
CAT_ESCOLARIDAD = "escolaridad.csv"
CAT_EDO_CIVIL = "estado_civil.csv"

# ICD-10 (catálogo original) y bridge ya generado (está en Output según tu screenshot)
ICD_SUIC_CAT = "ICD-10_suicidio.csv"
ICD_SUIC_BRIDGE = DIR_OUTPUT / "ICD-10_suicidio_bridge.csv"

# -------------------------
# Parámetros suicidio
# -------------------------
ICD10_INI = "X60"
ICD10_FIN = "X84"
RURAL_TLOC = {1, 2, 99}

# Outputs
OUT_POB_ZM_2024 = DIR_OUTPUT / "ZonaMetro_Pob2024.csv"
OUT_CLEAN = DIR_OUTPUT / "Suicidios_Clean.csv"
OUT_ANALISIS_ZM = DIR_OUTPUT / "AnalisisVentanas8d_ZM.csv"

# -------------------------
# Lecturas (solo aquí)
# -------------------------
df_edr = pd.read_csv(Dir_Defunciones / ARCHIVO_CONJ_DEF, encoding="latin1", low_memory=False)

df_bridge = pd.read_csv(Dir_Metropolis / ARCHIVO_Metropoli_Bridge, encoding="utf-8")
df_metro_car = pd.read_csv(Dir_Metropolis / ARCHIVO_Carateristicas_Metro, encoding="utf-8")
df_proy = pd.read_csv(Dir_Metropolis / ARCHIVO_Proyecciones_Demograficas, encoding="latin1")

df_paises = pd.read_csv(DIR_CATALOGOS / CAT_PAISES, encoding="utf-8")
df_ocup = pd.read_csv(DIR_CATALOGOS / CAT_OCUPACION, encoding="utf-8")
df_esco = pd.read_csv(DIR_CATALOGOS / CAT_ESCOLARIDAD, encoding="utf-8")
df_eciv = pd.read_csv(DIR_CATALOGOS / CAT_EDO_CIVIL, encoding="utf-8")

# Bridge ICD ya generado (si no existe, aquí fallará y sabrás que te falta correr el script del bridge)
df_icd_bridge = pd.read_csv(ICD_SUIC_BRIDGE, encoding="utf-8")




In [8]:
# ================================================================
# Paso 1.1 - Población 2024 por Zona Metropolitana (NO imports/lecturas)
# ================================================================

# -------------------------
# 1) Estandarización de claves (NO cambiar nombres de columnas)
# -------------------------
df_metro_car = df_metro_car.copy()
df_bridge = df_bridge.copy()
df_proy = df_proy.copy()

df_metro_car["clave_metropoli"] = df_metro_car["clave_metropoli"].astype(str).str.strip()
df_metro_car["nombre"] = df_metro_car["nombre"].astype(str).str.strip()
df_metro_car["poblacion"] = pd.to_numeric(df_metro_car["poblacion"], errors="coerce")

df_bridge["Cve_Metro"] = df_bridge["Cve_Metro"].astype(str).str.strip()
df_bridge["Cve_mun"]   = df_bridge["Cve_mun"].astype(str).str.strip().str.zfill(5)

df_proy["CLAVE"] = df_proy["CLAVE"].astype(str).str.strip().str.zfill(5)
df_proy["AÑO"]   = pd.to_numeric(df_proy["AÑO"], errors="coerce")
df_proy["POB_MIT_MUN"] = pd.to_numeric(df_proy["POB_MIT_MUN"], errors="coerce")

# -------------------------
# 2) Filtrar 2024
# -------------------------
df_proy_2024 = (
    df_proy.loc[df_proy["AÑO"] == 2024, ["CLAVE", "AÑO", "POB_MIT_MUN"]]
    .dropna(subset=["CLAVE", "POB_MIT_MUN"])
    .copy()
)

# -------------------------
# 3) Join municipio -> ZM y agregación por ZM
# -------------------------
df_mun_metro_2024 = df_bridge.merge(
    df_proy_2024,
    left_on="Cve_mun",
    right_on="CLAVE",
    how="left",
    validate="many_to_one"
)

pob2024_zm = (
    df_mun_metro_2024
    .groupby("Cve_Metro", as_index=False)["POB_MIT_MUN"]
    .sum()
    .rename(columns={"Cve_Metro": "clave_metropoli", "POB_MIT_MUN": "poblacion2024"})
)

df_zm_pob = df_metro_car.merge(
    pob2024_zm,
    on="clave_metropoli",
    how="left",
    validate="one_to_one"
)

df_zm_pob = df_zm_pob.rename(columns={"poblacion": "poblacion2020"})
df_zm_pob = df_zm_pob[["clave_metropoli", "nombre", "poblacion2020", "poblacion2024"]].copy()

# ================================================================
# Export - Población ZM 2024 (NO imports/lecturas)
# ================================================================
OUT_POB_ZM_2024.parent.mkdir(parents=True, exist_ok=True)
df_zm_pob.to_csv(OUT_POB_ZM_2024, index=False, encoding="utf-8")
OUT_POB_ZM_2024

WindowsPath('D:/Contenido/2025_12/Suicidios_Navidad/Repositorio/Output/ZonaMetro_Pob2024.csv')

In [8]:
import pandas as pd
from pathlib import Path
import re


RUTA_OUT = DIR_OUTPUT / "ICD-10_suicidio_bridge.csv"

df = pd.read_csv(RUTA_ICD, encoding="utf-8")

# Detectar columnas
cols_lower = {c.lower(): c for c in df.columns}
col_codigo = cols_lower.get("cve") or cols_lower.get("codigo") or cols_lower.get("code")
col_desc   = cols_lower.get("descrip") or cols_lower.get("descripcion") or cols_lower.get("desc")

if col_codigo is None or col_desc is None:
    raise ValueError(f"No pude detectar columnas código/desc. Columnas: {list(df.columns)}")

df = df.rename(columns={col_codigo: "Codigo", col_desc: "Descripcion"}).copy()

# ================================================================
# NORMALIZACIÓN CLAVE (arregla ESCUELAS", OTRAS...)
# ================================================================
desc = (
    df["Descripcion"].astype(str).str.upper().str.strip()
    .str.replace('"', '', regex=False)                 # quita comillas
    .str.replace(r"\s+", " ", regex=True)             # normaliza espacios
    .str.replace(r"\s*,\s*", ",", regex=True)         # coma sin espacios alrededor
)

PREF_ENV     = "ENVENENAMIENTO AUTOINFLIGIDO INTENCIONALMENTE POR,Y EXPOSICION A"
PREF_LES_POR = "LESION AUTOINFLIGIDA INTENCIONALMENTE POR"
PREF_LES_AL  = "LESION AUTOINFLIGIDA INTENCIONALMENTE AL"

is_env = desc.str.startswith(PREF_ENV)
is_les = desc.str.startswith(PREF_LES_POR) | desc.str.startswith(PREF_LES_AL)

tipo = pd.Series(pd.NA, index=df.index, dtype="object")
tipo.loc[is_env] = "Envenenamiento"
tipo.loc[is_les] = "Lesión"

rest = desc.copy()
rest.loc[is_env] = rest.loc[is_env].str[len(PREF_ENV):]
rest.loc[desc.str.startswith(PREF_LES_POR)] = rest.loc[desc.str.startswith(PREF_LES_POR)].str[len(PREF_LES_POR):]
rest.loc[desc.str.startswith(PREF_LES_AL)]  = rest.loc[desc.str.startswith(PREF_LES_AL)].str[len(PREF_LES_AL):]
rest = rest.str.strip().str.lstrip(",").str.strip()

# Lugares válidos (sin comillas, sin espacios tras coma)
LUGARES_VALIDOS = sorted([
    "VIVIENDA",
    "INSTITUCION RESIDENCIAL",
    "ESCUELAS,OTRAS INSTITUCIONES Y AREAS ADMINISTRATIVAS PUBLICAS",
    "AREAS DE DEPORTE Y ATLETISMO",
    "CALLES Y CARRETERAS",
    "COMERCIO Y AREA DE SERVICIOS",
    "AREA INDUSTRIAL Y DE LA CONSTRUCCION",
    "GRANJA",
    "OTRO LUGAR ESPECIFICADO",
    "LUGAR NO ESPECIFICADO",
], key=len, reverse=True)

def extraer_modo_lugar(txt: str):
    if not isinstance(txt, str) or txt.strip() == "":
        return (pd.NA, pd.NA)
    t = txt.strip()
    for lug in LUGARES_VALIDOS:
        if t.endswith(lug):
            modo = t[:-len(lug)].rstrip()
            modo = re.sub(r"[,\s]+$", "", modo).strip()
            return (modo if modo else pd.NA, lug)
    # fallback
    if "," in t:
        a, b = t.rsplit(",", 1)
        return (a.strip() if a.strip() else pd.NA, b.strip() if b.strip() else pd.NA)
    return (t, pd.NA)

modo_lugar = rest.apply(extraer_modo_lugar)
modo  = modo_lugar.apply(lambda x: x[0])
lugar = modo_lugar.apply(lambda x: x[1])

df_bridge = pd.DataFrame({
    "Codigo": df["Codigo"].astype(str).str.strip(),
    "Tipo": tipo,
    "Modo": modo,
    "Lugar": lugar
})

df_bridge.to_csv(RUTA_OUT, index=False, encoding="utf-8")
RUTA_OUT


WindowsPath('D:/Contenido/2025_12/Suicidios_Navidad/Repositorio/Output/ICD-10_suicidio_bridge.csv')

In [5]:
# ================================================================
# Paso 2 - Limpieza (NO imports, NO lecturas)
# ================================================================

# -------------------------
# Helpers
# -------------------------
def _norm_icd10(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.upper()
         .str.strip()
         .str.replace(r"\.", "", regex=True)
         .str.replace(r"\s+", "", regex=True)
    )

def _is_between_icd10(code_norm: pd.Series, ini: str, fin: str) -> pd.Series:
    c3 = code_norm.str.slice(0, 3)  # XNN
    return (c3 >= ini) & (c3 <= fin)

def _mk_cve_mun(ent: pd.Series, mun: pd.Series) -> pd.Series:
    return ent.astype(str).str.zfill(2) + mun.astype(str).str.zfill(3)

def _parse_edad(edad_raw: pd.Series) -> pd.Series:
    s = edad_raw.astype(str).str.strip().str.zfill(4)
    pref = s.str[0]
    out = pd.Series([pd.NA] * len(s), index=s.index, dtype="Int64")
    mask_ok = pref.eq("4")
    years = pd.to_numeric(s[mask_ok].str[1:4], errors="coerce")
    out.loc[mask_ok] = years.astype("Int64")
    return out

def _map_from_catalog(df_cat: pd.DataFrame, key_col="cve", val_col="descrip") -> dict:
    tmp = df_cat.copy()
    tmp[key_col] = tmp[key_col].astype(str).str.strip()
    tmp[val_col] = tmp[val_col].astype(str).str.strip()
    return dict(zip(tmp[key_col], tmp[val_col]))

def _zona_tipo_no_metro(tloc_series: pd.Series, rural_set: set) -> pd.Series:
    t = pd.to_numeric(tloc_series, errors="coerce")
    def f(x):
        if pd.isna(x):
            return "no_metropolitana_sin_tloc"
        return "rural" if int(x) in rural_set else "urbana_no_metropolitana"
    return t.apply(f)

def _desc_1_2_8_9(x):
    return {1: "si", 2: "no", 8: "no_aplica", 9: "no_especificado"}.get(x, pd.NA)

# -------------------------
# 0) Normalizar insumos mínimos
# -------------------------
# Bridge municipal->ZM (NO lo reutilizamos mutado; guardamos base)
df_bridge_base = df_bridge.copy()
df_bridge_base["Cve_Metro"] = df_bridge_base["Cve_Metro"].astype(str).str.strip()
df_bridge_base["Cve_mun"]   = df_bridge_base["Cve_mun"].astype(str).str.strip().str.zfill(5)

# Bridge ICD-10 suicidio
df_icd_bridge = df_icd_bridge.copy()
df_icd_bridge["Codigo"] = df_icd_bridge["Codigo"].astype(str).str.strip().str.upper()
# (Modo/Lugar/Tipo como texto limpio)
for c in ["Tipo", "Modo", "Lugar"]:
    if c in df_icd_bridge.columns:
        df_icd_bridge[c] = df_icd_bridge[c].astype(str).str.strip()

# Catálogos a dict
map_paises = _map_from_catalog(df_paises, "cve", "descrip")
map_ocup = _map_from_catalog(df_ocup, "cve", "descrip")
map_esco = _map_from_catalog(df_esco, "CVE", "DESCRIP")
map_eciv = _map_from_catalog(df_eciv, "CVE", "DESCRIP")

# -------------------------
# 1) Filtrado suicidio X60–X84 (todo el año) + columnas relevantes
# -------------------------
df_edr = df_edr.copy()
df_edr["causa_def_norm"] = _norm_icd10(df_edr["causa_def"])
mask_suic = _is_between_icd10(df_edr["causa_def_norm"], ICD10_INI, ICD10_FIN)

cols_keep = [
    # causa
    "causa_def", "causa_def_norm",
    # tiempo
    "dia_ocurr", "mes_ocurr", "horas",
    # ocurrencia
    "ent_ocurr", "mun_ocurr", "tloc_ocurr",
    # residencia
    "ent_resid", "mun_resid",
    # lesión
    "ent_ocules", "mun_ocules",
    # socio-demo
    "sexo", "afromex", "conindig", "nacionalidad", "nacesp_cve",
    "edad", "Ocupacion", "escolarida", "edo_civil",
]
cols_keep = [c for c in cols_keep if c in df_edr.columns]

df_suic = df_edr.loc[mask_suic, cols_keep].copy()

# -------------------------
# 1.1) Añadir bridge ICD (Codigo, Tipo, Modo, Lugar)
# -------------------------
# Nota: tu causa_def es C(4): letra + 3 números (ej. X700). Usamos causa_def_norm.
# Si causa_def_norm trae algo más largo, cortamos a 4 para empatar con Codigo.
df_suic["Codigo"] = df_suic["causa_def_norm"].str.slice(0, 4)

df_suic = df_suic.merge(
    df_icd_bridge[["Codigo", "Tipo", "Modo", "Lugar"]],
    on="Codigo",
    how="left",
    validate="many_to_one"
)

# -------------------------
# 2) Join por ZM (residencia / ocurrencia / lesión)
# -------------------------
df_suic["Cve_mun_ocurr"] = _mk_cve_mun(df_suic["ent_ocurr"], df_suic["mun_ocurr"])
df_suic["Cve_mun_resid"] = _mk_cve_mun(df_suic["ent_resid"], df_suic["mun_resid"])

if ("ent_ocules" in df_suic.columns) and ("mun_ocules" in df_suic.columns):
    df_suic["Cve_mun_lesion"] = _mk_cve_mun(df_suic["ent_ocules"], df_suic["mun_ocules"])
else:
    df_suic["Cve_mun_lesion"] = pd.NA

# Ocurrencia → metro
df_suic = df_suic.merge(
    df_bridge_base.rename(columns={"Cve_mun": "Cve_mun_ocurr", "Cve_Metro": "Cve_Metro_ocurr"}),
    on="Cve_mun_ocurr",
    how="left",
    validate="many_to_one"
)

# Residencia → metro
df_suic = df_suic.merge(
    df_bridge_base.rename(columns={"Cve_mun": "Cve_mun_resid", "Cve_Metro": "Cve_Metro_resid"}),
    on="Cve_mun_resid",
    how="left",
    validate="many_to_one"
)

# Lesión → metro
df_suic = df_suic.merge(
    df_bridge_base.rename(columns={"Cve_mun": "Cve_mun_lesion", "Cve_Metro": "Cve_Metro_lesion"}),
    on="Cve_mun_lesion",
    how="left",
    validate="many_to_one"
)

# Tipo de zona
mask_no_metro_ocurr = df_suic["Cve_Metro_ocurr"].isna()
df_suic.loc[~mask_no_metro_ocurr, "TipoZona_ocurr"] = "metropolitana"
df_suic.loc[mask_no_metro_ocurr, "TipoZona_ocurr"] = _zona_tipo_no_metro(df_suic.loc[mask_no_metro_ocurr, "tloc_ocurr"], RURAL_TLOC)

df_suic["TipoZona_resid"] = df_suic["Cve_Metro_resid"].apply(lambda x: "metropolitana" if pd.notna(x) else "no_metropolitana")
df_suic["TipoZona_lesion"] = df_suic["Cve_Metro_lesion"].apply(lambda x: "metropolitana" if pd.notna(x) else "no_metropolitana")

# Banderas útiles
df_suic["ocurr_fuera_metro_resid"] = (
    df_suic["Cve_Metro_resid"].notna() &
    (df_suic["Cve_Metro_ocurr"].fillna("NA") != df_suic["Cve_Metro_resid"])
)

df_suic["lesion_fuera_metro_resid"] = (
    df_suic["Cve_Metro_resid"].notna() &
    (df_suic["Cve_Metro_lesion"].fillna("NA") != df_suic["Cve_Metro_resid"])
)

# -------------------------
# 3) Homogenization: columnas espejo (código + descripción)
# -------------------------
# sexo
df_suic["sexo_code"] = pd.to_numeric(df_suic.get("sexo"), errors="coerce")
df_suic["sexo_desc"] = df_suic["sexo_code"].map({1: "hombre", 2: "mujer", 9: "no_especificado"})

# afromex / conindig
for col in ["afromex", "conindig"]:
    if col in df_suic.columns:
        code = pd.to_numeric(df_suic[col], errors="coerce")
        df_suic[col + "_code"] = code
        df_suic[col + "_desc"] = code.apply(_desc_1_2_8_9)

# nacionalidad
if "nacionalidad" in df_suic.columns:
    ncode = pd.to_numeric(df_suic["nacionalidad"], errors="coerce")
    df_suic["nacionalidad_code"] = ncode
    df_suic["nacionalidad_desc"] = ncode.map({1: "mexicana", 2: "extranjera", 9: "no_especificado"})

# país nacimiento (si es extranjera, esto te sirve mucho)
if "nacesp_cve" in df_suic.columns:
    df_suic["nacesp_cve_code"] = df_suic["nacesp_cve"].astype(str).str.strip()
    df_suic["nacesp_pais_desc"] = df_suic["nacesp_cve_code"].map(map_paises)

# edad
if "edad" in df_suic.columns:
    df_suic["edad_raw"] = df_suic["edad"]
    df_suic["edad_anios"] = _parse_edad(df_suic["edad"])

# ocupación
if "Ocupacion" in df_suic.columns:
    df_suic["ocupacion_code"] = df_suic["Ocupacion"].astype(str).str.strip()
    df_suic["ocupacion_desc"] = df_suic["ocupacion_code"].map(map_ocup)

# escolaridad
if "escolarida" in df_suic.columns:
    df_suic["escolarida_code"] = df_suic["escolarida"].astype(str).str.strip()
    df_suic["escolarida_desc"] = df_suic["escolarida_code"].map(map_esco)

# estado civil
if "edo_civil" in df_suic.columns:
    df_suic["edo_civil_code"] = df_suic["edo_civil"].astype(str).str.strip()
    df_suic["edo_civil_desc"] = df_suic["edo_civil_code"].map(map_eciv)

# día/mes/hora
if "dia_ocurr" in df_suic.columns:
    d = pd.to_numeric(df_suic["dia_ocurr"], errors="coerce")
    df_suic["dia_ocurr_code"] = d
    df_suic["dia"] = d.where((d >= 1) & (d <= 31), pd.NA).astype("Int64")

if "mes_ocurr" in df_suic.columns:
    m = pd.to_numeric(df_suic["mes_ocurr"], errors="coerce")
    df_suic["mes_ocurr_code"] = m
    df_suic["mes"] = m.where((m >= 1) & (m <= 12), pd.NA).astype("Int64")

if "horas" in df_suic.columns:
    h = pd.to_numeric(df_suic["horas"], errors="coerce")
    df_suic["hora_code"] = h
    df_suic["hora"] = h.where((h >= 0) & (h <= 23), pd.NA).astype("Int64")

# -------------------------
# 4) Output clean (df listo)
# -------------------------
df_clean = df_suic.copy()

OUT_CLEAN.parent.mkdir(parents=True, exist_ok=True)
df_clean.to_csv(OUT_CLEAN, index=False, encoding="utf-8")
OUT_CLEAN


WindowsPath('D:/Contenido/2025_12/Suicidios_Navidad/Repositorio/Output/Suicidios_Clean.csv')

In [9]:
# ================================================================
# Paso 3.1 - Ventanas móviles de 8 días (Nacional + por ZM)
# ================================================================

BASE_ZM = "resid"  # "resid" o "ocurr"
col_metro = "Cve_Metro_resid" if BASE_ZM == "resid" else "Cve_Metro_ocurr"

NAV_START = (12, 24)
NAV_END   = (12, 31)
YEAR_FAKE = 2024  # año fijo para ordenar; EDR 2024

# -------------------------
# Helpers
# -------------------------
def _make_date(df, year=2024):
    d = df.copy()
    d = d[d["mes"].notna() & d["dia"].notna()].copy()
    d["mes_i"] = d["mes"].astype(int)
    d["dia_i"] = d["dia"].astype(int)
    d = d[(d["mes_i"] >= 1) & (d["mes_i"] <= 12) & (d["dia_i"] >= 1) & (d["dia_i"] <= 31)].copy()
    d["fecha"] = pd.to_datetime({"year": year, "month": d["mes_i"], "day": d["dia_i"]}, errors="coerce")
    return d[d["fecha"].notna()].copy()

def _top_k_windows(daily_counts, k=3, window=8):
    s = daily_counts.sort_index().astype(int)
    roll = s.rolling(window=window, min_periods=window).sum().dropna()
    if roll.empty:
        return []
    top_end = roll.sort_values(ascending=False).head(k).index
    out = []
    for end in top_end:
        start = end - pd.Timedelta(days=window - 1)
        out.append({"start": start, "end": end, "count": int(roll.loc[end])})
    return out

def _fixed_window_sum(daily_counts, start_md, end_md, year=2024):
    start = pd.Timestamp(year=year, month=start_md[0], day=start_md[1])
    end   = pd.Timestamp(year=year, month=end_md[0], day=end_md[1])
    s = daily_counts.sort_index().astype(int)
    return int(s.loc[(s.index >= start) & (s.index <= end)].sum())

def _rank_of_window(top_list, start_dt, end_dt):
    # rank 1..len(top_list) si coincide EXACTO con top window. Si no, 0.
    for i, w in enumerate(top_list, start=1):
        if (w["start"] == start_dt) and (w["end"] == end_dt):
            return i
    return 0

# -------------------------
# Nacional: primer print (top1 vs navidad)
# -------------------------
df_base = _make_date(df_clean.copy(), year=YEAR_FAKE)
daily_nat = df_base.groupby("fecha").size()

top2_nat = _top_k_windows(daily_nat, k=2, window=8)
nav_start_dt = pd.Timestamp(YEAR_FAKE, NAV_START[0], NAV_START[1])
nav_end_dt   = pd.Timestamp(YEAR_FAKE, NAV_END[0], NAV_END[1])
nav_nat = _fixed_window_sum(daily_nat, NAV_START, NAV_END, year=YEAR_FAKE)

if len(top2_nat) >= 2:
    is_nav_top1_nat = (nav_nat == top2_nat[0]["count"])
    print(
        f"A nivel nacional este rango de fechas [{'si' if is_nav_top1_nat else 'no'}] "
        f"fue cuando más defunciones por este motivo se registraron con {top2_nat[0]['count']}. "
        f"El segundo rango fue de {top2_nat[1]['start'].strftime('%d/%m')} a {top2_nat[1]['end'].strftime('%d/%m')} "
        f"con {top2_nat[1]['count']} defunciones."
    )
elif len(top2_nat) == 1:
    is_nav_top1_nat = (nav_nat == top2_nat[0]["count"])
    print(
        f"A nivel nacional este rango de fechas [{'si' if is_nav_top1_nat else 'no'}] "
        f"fue cuando más defunciones por este motivo se registraron con {top2_nat[0]['count']}. "
        "No hubo un segundo rango disponible."
    )
else:
    print("No se pudieron calcular ventanas nacionales de 8 días (fechas insuficientes o inválidas).")

# -------------------------
# Por ZM: construir daily counts
# -------------------------
df_m = df_base[df_base[col_metro].notna()].copy()
g = df_m.groupby([col_metro, "fecha"]).size().rename("n").reset_index()

def _summarize_one_metro(df_one):
    s = df_one.set_index("fecha")["n"].sort_index()
    top3 = _top_k_windows(s, k=3, window=8)
    total = int(s.sum())
    nav_cnt = _fixed_window_sum(s, NAV_START, NAV_END, year=YEAR_FAKE)

    def get_i(i):
        if i < len(top3):
            return top3[i]["start"], top3[i]["end"], top3[i]["count"]
        return (pd.NaT, pd.NaT, pd.NA)

    t1s, t1e, t1c = get_i(0)
    t2s, t2e, t2c = get_i(1)
    t3s, t3e, t3c = get_i(2)

    nav_rank = _rank_of_window(top3, nav_start_dt, nav_end_dt)  # 1/2/3 o 0

    return pd.Series({
        "Suicidios_totales": total,
        "Navidad_24_31_dic": nav_cnt,
        "Navidad_rank": nav_rank,
        "Navidad_es_top1": (nav_rank == 1),

        "Top1_inicio": t1s, "Top1_fin": t1e, "Top1_suicidios": t1c,
        "Top2_inicio": t2s, "Top2_fin": t2e, "Top2_suicidios": t2c,
        "Top3_inicio": t3s, "Top3_fin": t3e, "Top3_suicidios": t3c,
    })

df_metro_windows = (
    g.groupby(col_metro, as_index=True)
     .apply(_summarize_one_metro)
     .reset_index()
     .rename(columns={col_metro: "Cve_Metro"})
)

# -------------------------
# Join con nombre/población (desde df_zm_pob creado en Paso 1.1)
# -------------------------
df_meta = df_zm_pob.copy()
df_meta = df_meta.rename(columns={"clave_metropoli": "Cve_Metro", "nombre": "Nombre", "poblacion2024": "Poblacion"})
df_meta["Cve_Metro"] = df_meta["Cve_Metro"].astype(str).str.strip()

df_out = df_metro_windows.merge(
    df_meta[["Cve_Metro", "Nombre", "Poblacion"]],
    on="Cve_Metro",
    how="left",
    validate="one_to_one"
)

df_out["Tasa_100k"] = (df_out["Suicidios_totales"] / df_out["Poblacion"]) * 100_000

# -------------------------
# Exportar TODAS las ZM
# -------------------------
# Orden de columnas
cols = [
    "Cve_Metro", "Nombre", "Poblacion",
    "Suicidios_totales", "Tasa_100k",
    "Navidad_24_31_dic", "Navidad_rank", "Navidad_es_top1",
    "Top1_inicio", "Top1_fin", "Top1_suicidios",
    "Top2_inicio", "Top2_fin", "Top2_suicidios",
    "Top3_inicio", "Top3_fin", "Top3_suicidios",
]
df_out = df_out[[c for c in cols if c in df_out.columns]].copy()

df_out.to_csv(OUT_ANALISIS_ZM, index=False, encoding="utf-8")
OUT_ANALISIS_ZM



A nivel nacional este rango de fechas [no] fue cuando más defunciones por este motivo se registraron con 256. El segundo rango fue de 19/04 a 26/04 con 255 defunciones.


C:\Users\Juan Pablo\AppData\Local\Temp\ipykernel_5236\2110503537.py:114: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_summarize_one_metro)


WindowsPath('D:/Contenido/2025_12/Suicidios_Navidad/Repositorio/Output/AnalisisVentanas8d_ZM.csv')

In [10]:
# ================================================================
# Paso 3.2 - ¿Navidad es outlier? (8 días) Nacional + Grupos + ZM
# Puede leer aquí archivos sin re-correr el bloque 0.
# ================================================================

import pandas as pd
import numpy as np
from pathlib import Path

# -------------------------
# Rutas (ajusta si tu DIR_REPO ya existe en memoria)
# -------------------------
try:
    DIR_REPO
except NameError:
    DIR_REPO = Path(r"D:\Contenido\2025_12\Suicidios_Navidad\Repositorio")

DIR_OUTPUT = DIR_REPO / "Output"
RUTA_CLEAN = DIR_OUTPUT / "Suicidios_Clean.csv"
RUTA_POB_ZM = DIR_OUTPUT / "ZonaMetro_Pob2024.csv"

OUT_NACIONAL = DIR_OUTPUT / "OutlierNavidad_Nacional.csv"
OUT_GRUPOS = DIR_OUTPUT / "OutlierNavidad_Grupos_Nacional.csv"
OUT_ZM = DIR_OUTPUT / "OutlierNavidad_ZM.csv"
OUT_ZM_GRUPOS = DIR_OUTPUT / "OutlierNavidad_ZM_Grupos.csv"

# -------------------------
# Parámetros ventana
# -------------------------
WINDOW = 8
YEAR_FAKE = 2024
NAV_START = (12, 24)
NAV_END = (12, 31)
nav_start_dt = pd.Timestamp(YEAR_FAKE, NAV_START[0], NAV_START[1])
nav_end_dt   = pd.Timestamp(YEAR_FAKE, NAV_END[0], NAV_END[1])

# Base geográfica (elige una)
BASE_ZM = "resid"   # "resid" o "ocurr"
COL_ZM = "Cve_Metro_resid" if BASE_ZM == "resid" else "Cve_Metro_ocurr"

# -------------------------
# Cargar df_clean si no existe
# -------------------------
if "df_clean" not in globals():
    df_clean = pd.read_csv(RUTA_CLEAN, encoding="utf-8", low_memory=False)

# Población ZM (para tasa total; opcional)
df_pob = None
if RUTA_POB_ZM.exists():
    df_pob = pd.read_csv(RUTA_POB_ZM, encoding="utf-8")
    df_pob["clave_metropoli"] = df_pob["clave_metropoli"].astype(str).str.strip()
    if "poblacion2024" in df_pob.columns:
        df_pob["poblacion2024"] = pd.to_numeric(df_pob["poblacion2024"], errors="coerce")

# -------------------------
# Helpers
# -------------------------
def make_date(df, year=2024):
    d = df.copy()
    # usa columnas ya limpias si existen
    if "mes" in d.columns and "dia" in d.columns:
        m = pd.to_numeric(d["mes"], errors="coerce")
        di = pd.to_numeric(d["dia"], errors="coerce")
    else:
        m = pd.to_numeric(d["mes_ocurr"], errors="coerce")
        di = pd.to_numeric(d["dia_ocurr"], errors="coerce")

    d = d.assign(_mes=m, _dia=di)
    d = d[d["_mes"].between(1, 12) & d["_dia"].between(1, 31)].copy()

    d["fecha"] = pd.to_datetime(
        {"year": year, "month": d["_mes"].astype(int), "day": d["_dia"].astype(int)},
        errors="coerce"
    )
    d = d[d["fecha"].notna()].copy()
    return d

def daily_counts(df, group_cols=None):
    if group_cols is None:
        return df.groupby("fecha").size().sort_index()
    return df.groupby(group_cols + ["fecha"]).size().rename("n").reset_index()

def rolling_windows_series(daily_series, window=8):
    s = daily_series.sort_index().astype(int)
    roll = s.rolling(window=window, min_periods=window).sum().dropna()
    # index = end date
    return roll

def fixed_window_sum(daily_series, start_dt, end_dt):
    s = daily_series.sort_index().astype(int)
    return int(s.loc[(s.index >= start_dt) & (s.index <= end_dt)].sum())

def robust_z_mad(values, x):
    """
    values: array-like de conteos (todas las ventanas)
    x: conteo en navidad
    Retorna robust z usando MAD. Si MAD=0, regresa NaN.
    """
    v = np.asarray(values, dtype=float)
    med = np.nanmedian(v)
    mad = np.nanmedian(np.abs(v - med))
    if mad == 0 or np.isnan(mad):
        return np.nan
    return (x - med) / (1.4826 * mad)

def percentile_of_x(values, x):
    v = np.asarray(values, dtype=float)
    v = v[~np.isnan(v)]
    if len(v) == 0:
        return np.nan
    return float((v <= x).mean())

def summarize_outlier_from_daily(daily_series, label="total"):
    """
    daily_series: Series index=fecha, values=conteos por día
    """
    roll = rolling_windows_series(daily_series, window=WINDOW)  # windows by end-date
    if roll.empty:
        return pd.Series({
            "grupo": label,
            "nav_count": 0,
            "nav_percentile": np.nan,
            "nav_robust_z": np.nan,
            "top1_count": np.nan,
            "top1_start": pd.NaT,
            "top1_end": pd.NaT,
            "n_windows": 0
        })

    # nav count fijo 24-31
    nav_cnt = fixed_window_sum(daily_series, nav_start_dt, nav_end_dt)

    # percentil / robust z vs todas las ventanas
    vals = roll.values
    nav_pct = percentile_of_x(vals, nav_cnt)
    nav_rz = robust_z_mad(vals, nav_cnt)

    # top1
    end_top1 = roll.idxmax()
    top1_cnt = int(roll.loc[end_top1])
    top1_start = end_top1 - pd.Timedelta(days=WINDOW - 1)
    top1_end = end_top1

    return pd.Series({
        "grupo": label,
        "nav_count": int(nav_cnt),
        "nav_percentile": nav_pct,
        "nav_robust_z": nav_rz,
        "top1_count": top1_cnt,
        "top1_start": top1_start,
        "top1_end": top1_end,
        "n_windows": int(len(roll))
    })

# -------------------------
# 1) Nacional (total)
# -------------------------
df_base = make_date(df_clean, year=YEAR_FAKE)
daily_nat = daily_counts(df_base)

res_nat = summarize_outlier_from_daily(daily_nat, label="nacional_total")

# Imprime el headline
# (si nav_percentile ~ 1.0 => navidad arriba; ~0.0 => abajo)
print(
    f"NACIONAL: Navidad (24-31 dic) tuvo {res_nat['nav_count']} casos. "
    f"Percentil={res_nat['nav_percentile']:.3f} (0=bajo, 1=alto), "
    f"RobustZ(MAD)={res_nat['nav_robust_z']:.2f}. "
    f"Top1 fue {int(res_nat['top1_count'])} del {res_nat['top1_start'].strftime('%d/%m')} al {res_nat['top1_end'].strftime('%d/%m')}."
)

pd.DataFrame([res_nat]).to_csv(OUT_NACIONAL, index=False, encoding="utf-8")

# -------------------------
# 2) Nacional por grupos
# -------------------------
# Definir grupos (ajusta a tus columnas disponibles)
df_base = df_base.copy()

# edad en bandas (usa edad_anios si existe; si no, intenta construirlo desde edad_raw)
if "edad_anios" in df_base.columns:
    edad = pd.to_numeric(df_base["edad_anios"], errors="coerce")
elif "edad_raw" in df_base.columns:
    # fallback: si tienes edad_raw en formato 4xxx
    s = df_base["edad_raw"].astype(str).str.strip().str.zfill(4)
    edad = pd.to_numeric(s.where(s.str.startswith("4")).str[1:4], errors="coerce")
else:
    edad = pd.Series(np.nan, index=df_base.index)

bins = [0, 14, 24, 34, 44, 54, 64, 74, 200]
labels = ["0-14","15-24","25-34","35-44","45-54","55-64","65-74","75+"]
df_base["edad_grupo"] = pd.cut(edad, bins=bins, labels=labels, right=True, include_lowest=True)

# columnas de grupo (solo si existen)
group_vars = []
for c in ["sexo_desc", "edad_grupo", "escolarida_desc", "conindig_desc"]:
    if c in df_base.columns:
        group_vars.append(c)

rows = []
for gv in group_vars:
    for val, df_sub in df_base.groupby(gv, dropna=False):
        daily = daily_counts(df_sub)
        label = f"{gv}={val}"
        rows.append(summarize_outlier_from_daily(daily, label=label))

df_grupos = pd.DataFrame(rows)
df_grupos.to_csv(OUT_GRUPOS, index=False, encoding="utf-8")

# ¿Cuál grupo está más "arriba" en navidad? (robust z mayor)
df_grupos_rank = df_grupos.dropna(subset=["nav_robust_z"]).sort_values("nav_robust_z", ascending=False)
if not df_grupos_rank.empty:
    top = df_grupos_rank.iloc[0]
    print(f"GRUPOS: Mayor outlier positivo en Navidad: {top['grupo']} | RobustZ={top['nav_robust_z']:.2f} | Percentil={top['nav_percentile']:.3f} | nav_count={int(top['nav_count'])}")

# -------------------------
# 3) Geografía: por ZM (todas)
# -------------------------
df_m = df_base[df_base[COL_ZM].notna()].copy()
df_m[COL_ZM] = df_m[COL_ZM].astype(str).str.strip()

g = daily_counts(df_m, group_cols=[COL_ZM])

def summarize_one_metro(df_one):
    s = df_one.set_index("fecha")["n"].sort_index()
    return summarize_outlier_from_daily(s, label="")  # label se rellena afuera

df_zm = (
    g.groupby(COL_ZM, as_index=True)
     .apply(summarize_one_metro)
     .reset_index()
     .rename(columns={COL_ZM: "Cve_Metro"})
)

# agregar nombre y población si existe
if df_pob is not None:
    meta = df_pob.rename(columns={
        "clave_metropoli": "Cve_Metro",
        "nombre": "Nombre",
        "poblacion2024": "Poblacion"
    }).copy()
    meta["Cve_Metro"] = meta["Cve_Metro"].astype(str).str.strip()

    df_zm = df_zm.merge(meta[["Cve_Metro","Nombre","Poblacion"]], on="Cve_Metro", how="left", validate="one_to_one")
    df_zm["tasa_anual_100k"] = (df_zm["nav_count"] * 0 + df_zm["top1_count"] * 0)  # placeholder si quieres
    # tasa anual no se define aquí; definimos tasa NAV y/o total anual si la quieres:
    # nota: summarize_outlier no devuelve total anual; si lo necesitas, se calcula aparte.

# Export map-ready
df_zm.to_csv(OUT_ZM, index=False, encoding="utf-8")

# -------------------------
# 4) Geografía por grupo (ZM x grupo) - útil para "qué grupo y dónde"
# -------------------------
# Esto puede crecer; lo limitamos a variables que sí existan y evitamos grupos ultra chicos.
MIN_EVENTS = 30  # umbral para estabilidad (ajústalo)

rows = []
for gv in group_vars:
    # contar eventos por ZM y grupo para filtrar
    ct = df_m.groupby([COL_ZM, gv]).size().rename("n_events").reset_index()
    ct = ct[ct["n_events"] >= MIN_EVENTS].copy()
    if ct.empty:
        continue

    # iterar combos válidos
    for _, r in ct.iterrows():
        zm = r[COL_ZM]
        val = r[gv]
        sub = df_m[(df_m[COL_ZM] == zm) & (df_m[gv] == val)].copy()
        daily = daily_counts(sub)
        out = summarize_outlier_from_daily(daily, label=f"{gv}={val}")
        out["Cve_Metro"] = zm
        out["variable"] = gv
        out["categoria"] = str(val)
        out["n_events"] = int(r["n_events"])
        rows.append(out)

df_zm_grupos = pd.DataFrame(rows)
df_zm_grupos.to_csv(OUT_ZM_GRUPOS, index=False, encoding="utf-8")

print("Exports:")
print(OUT_NACIONAL)
print(OUT_GRUPOS)
print(OUT_ZM)
print(OUT_ZM_GRUPOS)


NACIONAL: Navidad (24-31 dic) tuvo 187 casos. Percentil=0.384 (0=bajo, 1=alto), RobustZ(MAD)=-0.30. Top1 fue 256 del 15/04 al 22/04.
GRUPOS: Mayor outlier positivo en Navidad: escolarida_desc=Secundaria completa | RobustZ=1.48 | Percentil=0.877 | nav_count=65


C:\Users\Juan Pablo\AppData\Local\Temp\ipykernel_5236\1465254666.py:202: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for val, df_sub in df_base.groupby(gv, dropna=False):
C:\Users\Juan Pablo\AppData\Local\Temp\ipykernel_5236\1465254666.py:230: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_one_metro)
C:\Users\Juan Pablo\AppData\Local\Temp\ipykernel_5236\1465254666.py:261: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass ob

Exports:
D:\Contenido\2025_12\Suicidios_Navidad\Repositorio\Output\OutlierNavidad_Nacional.csv
D:\Contenido\2025_12\Suicidios_Navidad\Repositorio\Output\OutlierNavidad_Grupos_Nacional.csv
D:\Contenido\2025_12\Suicidios_Navidad\Repositorio\Output\OutlierNavidad_ZM.csv
D:\Contenido\2025_12\Suicidios_Navidad\Repositorio\Output\OutlierNavidad_ZM_Grupos.csv
